# Nashville Analytics Summit 2021

# Pathway to Predicting expensive providers for the BHHS using Python

# Defining the purpose:

In [ ]:
# Import key modules/packages

import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import pickle
import re ## Used for regular expressions

In [ ]:
# Modeling Imports
import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy

from sklearn.linear_model import LinearRegression, Ridge, Lasso, RidgeCV, LassoCV, ElasticNet
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Data Acquisiton:

In [ ]:
# Go to website to acquire data Center for Medicare and Medicaid Services
# 
df = pd.read_csv(r'C:\Users\JayRoy\Desktop\NAS 21\Data Sets\newdata2.csv', sep='|')

## Since this is Medicare data, some physicians do not participate i.e. accept Medicare payments, because it less than
## what they can be reimbursed for on a commercial claim

# Data Exploration, Profiling and Cleansing:

In [ ]:
df.head(10)

In [ ]:
df.shape

In [ ]:
df.head(10) 

In [ ]:
## Let's do some initial data profiling and look at the specific names of the variables in our dataset
list(df)
## What types of data do we have in this dataset ?
## What would be helpful to understand the dataset ? A: Data Dictionary
## Many of our fields are appended with "Rndrng_" or example 'Rndrng_Prvdr_First_Name' - What is the meaning of "Rndrng_" ?
## Few fields are appended with "Tot_" .... - What is the meaning of "Tot_" ?
## https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4120293/#:~:text=The%20log%20transformation%20is%2C%20arguably,normal%20or%20near%20normal%20distribution.


In [ ]:
df.iloc[1:2, 18:25]
## We will be interested in Tot_Srvcs,Tot_Mdcr_Pymt_Amt
## Interesting to see how much the difference between Submitted, Allowed and Paid amounts.

## Drug charges - Medicare Part D
df.iloc[0:1, 25:33]


In [ ]:
## What's the quality of the data ? - Remember G.I.G.O. OR R.I.R.O. ? 
## Missing data 
## Remove incomplete rows - Use case - enrollment data does not have right SSN in the dataset)
## Change Casing
## lots of NaN

In [ ]:
## Check to see if there are any providers in NOT paraticipating or empty values ?
## If so, remove NaN from Rndrng_Prvdr_Ent_Cd column
df[df.Rndrng_Prvdr_Mdcr_Prtcptg_Ind.isnull() == True]


In [ ]:
##df.drop(index=1053958, inplace=True)
df.drop(index=1028534, inplace=True)

In [ ]:
df[df.Rndrng_Prvdr_Mdcr_Prtcptg_Ind.isnull() == True]

In [ ]:
## Now that we know that the providers are participating in Medicare, let's look at the types of providers in our dataset.
## The variable is Rndrng_Prvdr_Type

In [ ]:
df.loc[:,"Rndrng_Prvdr_Type"] ## Using python slicing, we are only wanting to peek at one column so we can slice one column

In [ ]:
## Since Million plus records, we want to know how unique types of physicians are in our dataset 
print(pd.unique(df['Rndrng_Prvdr_Type']))

In [ ]:
n = len(pd.unique(df['Rndrng_Prvdr_Type']))

In [ ]:
print("No.of.unique specialty types :", n)

In [ ]:
## Alternatively you can use the code below.
df['Rndrng_Prvdr_Type'].nunique()

In [ ]:
## In our dataset, how many physicians are Endocrinology?
num_of_endo_docs =df["Rndrng_Prvdr_Type"].str.count("Endocrinology", re.I)

In [ ]:
num_of_endo_docs.value_counts()

In [ ]:
## In our dataset, how many physicians are Internal Medicine?

num_of_im_docs =df["Rndrng_Prvdr_Type"].str.count("Internal Medicine", re.I)

In [ ]:
## There are Internal Medicine physicians
num_of_im_docs.value_counts() 


In [ ]:
## Lets look at the how many physicians from highest to lowest.
df['Rndrng_Prvdr_Type'].value_counts()

In [ ]:
df['Rndrng_Prvdr_Type'].value_counts().hist()

In [ ]:
## Create a dataset of providers
##df_pt = pd.DataFrame(columns = df['Rndrng_Prvdr_Type'])
df_pt = df['Rndrng_Prvdr_Type']


##pd.Series(df_pt).value_counts().plot('bar')

df_pt

In [ ]:
## Create a dataset of providers

from collections import Counter
import pandas

In [ ]:
df_pt = df['Rndrng_Prvdr_Type']

In [ ]:
provider_counts = Counter(df_pt)

In [ ]:
# Creating a histogram shows use there are a few types of physicians that may give us indication of how certain physicians may be more costly?

df_hist = pandas.DataFrame.from_dict(provider_counts, orient='index')
df_hist.plot(kind='bar')

In [ ]:
df.loc[:,['Rndrng_Prvdr_Type','Tot_Srvcs','Tot_Mdcr_Pymt_Amt']]

In [ ]:
Creating heatmap to look for multicolinearity and correlations 
# between target(TOTAL_MEDICARE_PAYMENT_AMT) and all predictor variables

In [ ]:
df[df.Rndrng_Prvdr_Gndr.isnull() == True].index

In [ ]:
# Need additional feature engineering for dropping medical organizations(don't have a gender) and only selecting individuals
indiv  = df.drop(index=(df[df.Rndrng_Prvdr_Gndr.isnull() == True].index))

In [ ]:
indiv.shape
#indiv.Rndrng_Prvdr_Cntry != 'US'
#indiv.Rndrng_Prvdr_Cntry == 'US'# 986276

In [ ]:
indiv.Rndrng_Prvdr_Gndr

In [ ]:
indiv.Rndrng_Prvdr_Mdcr_Prtcptg_Ind

In [ ]:
# A big part of feature engingeering is being able to determine relevant fields
## In this case we need to remove doctors who dont participate in Medicare. 
indiv = indiv[indiv.Rndrng_Prvdr_Mdcr_Prtcptg_Ind != 'N']

In [ ]:
indiv.Rndrng_Prvdr_Mdcr_Prtcptg_Ind.value_counts()

In [ ]:
nan_fill_list = [
'Bene_Avg_Age',
'Bene_Age_75_84_Cnt',
'Bene_Age_GT_84_Cnt',
'Bene_Feml_Cnt',
'Bene_Male_Cnt',
'Bene_Race_Wht_Cnt',
'Bene_Race_Black_Cnt',
'Bene_Race_API_Cnt',
'Bene_Race_Hspnc_Cnt',
'Bene_Race_NatInd_Cnt',
'Bene_Race_Othr_Cnt',
'Bene_Dual_Cnt',
'Bene_Ndual_Cnt',
'Bene_CC_AF_Pct',
'Bene_CC_Alzhmr_Pct',
'Bene_CC_Asthma_Pct',
'Bene_CC_Cncr_Pct',
'Bene_CC_CHF_Pct',
'Bene_CC_CKD_Pct',
'Bene_CC_COPD_Pct',
'Bene_CC_Dprssn_Pct',
'Bene_CC_Dbts_Pct',
'Bene_CC_Hyplpdma_Pct',
'Bene_CC_Hyprtnsn_Pct',
'Bene_CC_IHD_Pct',
'Bene_CC_Opo_Pct',
'Bene_CC_RAOA_Pct',
'Bene_CC_Sz_Pct',
'Bene_CC_Strok_Pct',
##'Bene_Avg_Risk_Scre',
'Bene_Avg_Risk_Scre,,,,,,,,'    
]
    

In [ ]:
print(nan_fill_list)

In [ ]:
# replacing nulls in numerical columns with zeroes
indiv.loc[:,nan_fill_list] = indiv.loc[:,nan_fill_list].fillna(0)

In [ ]:
indiv.shape

In [ ]:
# removing unnecessary columns for next stage of analysis
clean_indiv = indiv.drop(columns=[
'Rndrng_NPI',
'Rndrng_Prvdr_Last_Org_Name',
'Rndrng_Prvdr_First_Name',
'Rndrng_Prvdr_MI',
'Rndrng_Prvdr_Crdntls',
'Rndrng_Prvdr_Ent_Cd',
'Rndrng_Prvdr_St1',
'Rndrng_Prvdr_St2',
'Rndrng_Prvdr_City',
##'Rndrng_Prvdr_State_Abrvtn',
'Rndrng_Prvdr_State_FIPS',
'Rndrng_Prvdr_Zip5',
'Rndrng_Prvdr_RUCA',
'Rndrng_Prvdr_RUCA_Desc',
'Rndrng_Prvdr_Cntry',
'Rndrng_Prvdr_Mdcr_Prtcptg_Ind'                                  
])

In [ ]:
clean_indiv.shape

In [ ]:
# Remove rows with Medicare values of 0 and keep only those actual payments from CMS
clean_indiv = clean_indiv[clean_indiv.Tot_Mdcr_Pymt_Amt != 0]

In [ ]:
clean_indiv.shape

In [ ]:
clean_indiv.head()

In [ ]:
# Dropping unnecessary or price-based columns
clean_df = clean_indiv.drop(columns = [
'Bene_Avg_Risk_Scre,,,,,,,,',
'Tot_Sbmtd_Chrg',
'Med_Tot_HCPCS_Cds',
'Med_Tot_Srvcs',
'Drug_Tot_HCPCS_Cds',
'Drug_Tot_Srvcs',
'Drug_Sprsn_Ind',
'Med_Sprsn_Ind',
'Tot_Mdcr_Alowd_Amt',
'Tot_Mdcr_Stdzd_Amt',
'Drug_Tot_Benes',
'Drug_Sbmtd_Chrg',
'Drug_Mdcr_Alowd_Amt',
'Drug_Mdcr_Pymt_Amt',
'Drug_Mdcr_Stdzd_Amt',
'Med_Tot_Benes',
'Med_Sbmtd_Chrg',
'Med_Mdcr_Alowd_Amt',
'Med_Mdcr_Pymt_Amt',
'Med_Mdcr_Stdzd_Amt',
])

In [ ]:
# Shape before removing columns
print(clean_indiv.shape)

# Shape after removing columns
print(clean_df.shape)

# Model Building without Feature Engineering

In [ ]:
y=a+ bx

x=clean_df.TOTAL_SERVICES, y=clean_df.TOTAL_MEDICARE_PAYMENT_AMT




# Model Building without Feature Engineering

In [ ]:
plt.figure(figsize=(10,10))
sns.heatmap(clean_df.corr(), cmap='Oranges');

In [ ]:

clean_df.corr()

In [ ]:
### Insight: To perform feature engineering well, you must understand your data but also the complex industry business rules
## and there are an excessive amount of feature engineering required and deal with some quirkiness prior to doing any modeling.
